In [6]:
import pandas as pd
import numpy as np
import json
def _normalize_amount(row):
    """Normalize token amounts based on common decimal patterns."""
    amount=row['amount']
    symbol=row['assetSymbol']
    decimal_map={
        'USDC':6,
        'USDT':6,
        'DAI':18,
        'WETH':18,
        'WMATIC':18,
        'WBTC':8
    }
    decimals=decimal_map.get(symbol,18)
    return amount/(10**decimals)
def load_transactions(file_path:str)->pd.DataFrame:
    """Load data from JSON file"""
    print(f"Loading transactions from {file_path}")
    try:
        with open(file_path,'r') as f:
            data=json.load(f)
        transactions=[]
        for record in data:
            tx={
                'wallet':record.get('userWallet',''),
                'network':record.get('network',''),
                'protocol':record.get('protocol',''),
                'txHash':record.get('txHash',''),
                'timestamp':record.get('timestamp',''),
                'blockNumber':record.get('blockNumber',''),
                'action':record.get('action',''),
            }
            action_data=record.get('actionData',{})
            tx.update({
                'amount':action_data.get('amount','0'),
                'assetSymbol':action_data.get('assetSymbol','0'),
                'assetPriceUSD':action_data.get('assetPriceUSD','0'),
                'poolId':action_data.get('poolId','0'),
                'userId':action_data.get('userId','0'),
                'type':action_data.get('type',''),
            })
            if 'createdAt' in record and isinstance(record['createdAt'],dict):
                if '$date' in record['createdAt']:
                    tx['createdAt']=record['createdAt']['$date']
            transactions.append(tx)
        df=pd.DataFrame(transactions)
        df['timestamp']=pd.to_datetime(df['timestamp'],unit='s',errors='coerce')
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        df['assetPriceUSD'] = pd.to_numeric(df['assetPriceUSD'], errors='coerce')
        df['blockNumber'] = pd.to_numeric(df['blockNumber'], errors='coerce')
        df['amountUSD'] = df['amount'] * df['assetPriceUSD']
        df['normalizedAmount']=df.apply(_normalize_amount,axis=1)
        df['normalizedAmountUSD']=df['normalizedAmount']*df['assetPriceUSD']
        df=df[df['wallet'].notna() & (df['wallet']!='')]
        df=df[df['timestamp'].notna()]
        df=df[df['amount']>0]
        #print(df)
        #print(df.columns)
        #print(df.describe)
        return df
    except Exception as e:
        print(f'error loading transactions:{e}')
        return pd.DataFrame()


path= 'user-wallet-transactions.json'
transaction_df=load_transactions(path)

Loading transactions from user-wallet-transactions.json


In [7]:
transaction_df['wallet'].nunique()

3497

In [3]:
import json
with open('user-wallet-transactions.json','r') as f:
    data=json.load(f)

In [4]:
len(data)

100000

TypeError: list indices must be integers or slices, not str